In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
import seaborn as sns
import collections

# Type of the graph
(e.g., power law, small world, regular, sampled manifold)

In [2]:
actors_agg_adj = np.load("sparse_agg_actor_adj.npy")
actors_agg_df = pd.read_pickle("actors_agg_df.pkl")
actors_graph = nx.from_numpy_matrix(actors_agg_adj)

In [3]:
# Average clustering coefficient
clustering_coefficient = nx.average_clustering(actors_graph)

Thus, it means that the network is not a small world type of graph, because its clustering coefficient is not small

For the other types of graphs:

In [4]:
# power law

In [5]:
# regular graph

In [6]:
# sample manifold

# Properties of nodes
(e.g., clustering coefficient, modularity, centrality)

In [7]:
# Average clustering coefficient
print(clustering_coefficient)

0.6403268534042411


In [8]:
# Degree centrality sorted
degree_centrality = nx.degree_centrality(actors_graph)
sorted_centrality = sorted(
    degree_centrality.items(), key=lambda kv: kv[1], reverse=True
)

In [9]:
# Actors that have more dense connections
for node_id, centrality in sorted_centrality[:5]:
    print(actors_agg_df.loc[node_id, "actors"], centrality)

Britney Spears 0.7812182741116752
Melissa Joan Hart 0.7578680203045686
Orlando Jones 0.733502538071066
Nicholas Rowe 0.7324873096446701
Tom Selleck 0.7279187817258883


In [10]:
# Actors that does not have dense connections
for node_id, centrality in sorted_centrality[-5:]:
    print(actors_agg_df.loc[node_id, "actors"], centrality)

Ben Youcef 0.0040609137055837565
Caitlin Fitzgerald 0.003553299492385787
Kirby Heyborne 0.0015228426395939086
J.D. Williams 0.0010152284263959391
Toshirō Mifune 0.0010152284263959391


In [11]:
# modularity